In [ ]:
import requests 
import json
import time
from pprint import pprint 
import pandas as pd 
import numpy as np


In [ ]:
# читаем токен, предварительно записанный в файл token.txt, файл положить рядом с ноутбуком
with open('token.txt', 'r') as file_object:
    token = file_object.read().strip()

In [ ]:
 #Параметры sort
    #0 — сортировать по умолчанию (аналогично результатам поиска в полной версии сайта);
    #1 — сортировать по скорости роста;
    #2 — сортировать по отношению дневной посещаемости к количеству пользователей;
    #3 — сортировать по отношению количества лайков к количеству пользователей;
    #4 — сортировать по отношению количества комментариев к количеству пользователей;
    #5 — сортировать по отношению количества записей в обсуждениях к количеству пользователей.
        
# читаем токен, предварительно записанный в файл token.txt, файл положить рядом с ноутбуком
with open('token.txt', 'r') as file_object:
    token = file_object.read().strip()

def groups_search_query(key_word, token, sorting=0, type_group='group', count=500 ):
    ''' используем метод VK API groups.search
    https://dev.vk.com/method/groups.search    
    '''
    URL = 'https://api.vk.com/method/{}' 
    method = 'groups.search'   
    params = {
        'q': key_word,
        'access_token': token,
        'sort': sorting,
        'count': count,
        'type': type_group,
        'v':'5.131'
    }
    res = requests.get(URL.format(method), params).json() # собираем запрос сразу обрабатываем методом .json()
    result = res['response']['items'] 
    return result

key_word = 'авто' # выборка по интересующему нас слову
groups_search_df =pd.DataFrame(groups_search_query(key_word, token, count=10, sorting=2, type_group='group'))

groups_search_df.tail()



In [ ]:
groups_search_df.to_csv('groups_df', sep=',') # сохраняем промежуточные результаты

### Вариант использования прямого метода *groups.getMembers*:

In [ ]:
df = pd.read_csv('groups_df', sep=',')

def vk_groups_members(group_id):
    ''' function for applying vk.com/groups.getMembers
    https://dev.vk.com/method/groups.getMembers
    '''
    time.sleep(0.32)
    with open('token.txt', 'r') as file:
        token = file.read().strip()

    url = 'https://api.vk.com/method/{}' # Указываем адрес обращения
    method = 'groups.getMembers'
    params = {
        'access_token': token,
        'group_id': group_id,
        'count': 1,
        'offset': 0,
        'v': 5.131
        } 
    
    max_count_req = requests.get(url.format(method), params = params).json()
    try:
        max_count = max_count_req['response']['count'] 
    except:
        return np.NaN # при ограничении частоты запроса ВК возвращает ошибки для их поиска возвращаем np.NaN
    offset = 0
    res_df = list()
    count = 500
    while offset < max_count:
         
        params = {
        'access_token': token,
        'group_id': group_id,
        'sort': 2,
        'count': count,
        'offset': offset,
        'v': 5.131
        } 
        time.sleep(0.32)
        data = requests.get(url.format(method), params = params).json()
        res_df.extend(data['response']['items'])
        offset += count
    return res_df


df['members'] = df['id'].apply(lambda x: vk_groups_members(x)) #добавляем списки пользователей групп
df.to_csv('df_with_members.csv') # сохранение итогов


### Вариант использования прямого метода *execute groups.getMembers*

In [ ]:

df = pd.read_csv('groups_df', sep=',')
def vk_groups_members(group_id):
    ''' function for applying vk.com/groups.getMembers
    https://dev.vk.com/method/groups.getMembers
    '''
    with open('token.txt', 'r') as file: #берем токен из файла
        token = file.read().strip()
   
    time.sleep(0.2) #даже с такой задержкой иногда вылетает ошибка по частоте хотя должен выдерживать 0.08с в данномм скрипте
    url = "https://api.vk.com/method/execute?"
    method = 'groups.getMembers'
    params = {
        'access_token': token,
        'group_id': group_id,
        'count': 1,
        'offset': 0,
        'v': 5.131
        } 
    
    code = 'return [API.{}({})];'.format(method, params)
    data = dict(code=code, access_token=token, v='5.131')
    max_count_req = requests.post(url=url, data=data).json()
    try:
        max_count = max_count_req['response'][0]['count'] # смотрим количество записей для использования в цикле выгрузки пользователей
    except:
        max_count = max_count_req['response'] #ВК часто выдает ограничения по частоте использования методов причем иногда без описания в документации
        print(f'wrong way {max_count}') 
        return np.NaN  # позже можно все NaN обработать
    
    offset = 0
    resuslt_list = list()
    count = 500 # максимальный 
    while offset < max_count:
        # начиная с того места, где закончили на предыдущей итерации (offset) 
        
        params = {
        'access_token': token,
        'group_id': group_id,
        'sort': 2,
        'count': count,
        'offset': offset,
        'v': 5.131
        }
        code = 'return [API.{}({})];'.format(method, params)
        
        time.sleep(0.2)
        data = dict(code=code, access_token=token, v='5.131')
        json_to_list = requests.post(url=url, data=data).json()
        
        try:
            lst = list(json_to_list['response']['items'])
        except:
            pprint(f'wrong way list {lst}')
            return lst # если вылетела ошибка по частоте то частично сохраняем пользователей хотя правильнее было бы np.NaN вернуть
                
        resuslt_list.extend(lst)
        offset += count # переходим на следующий интервал пользователей
    return resuslt_list


df['members'] = df['id'].apply(lambda x: vk_groups_members(x))
df.to_csv('df_with_members_ex.csv')



In [ ]:

df.head()
